### 2019.9.24

## 知识碎片
    app.config 为Flask提供的一个字典，为内置配置机制，可以用来增加自己的键值对。

## 创建上下文管理器（类）
### 协议
    1.具有__enter__方法，当用with挂接时，自动调用__enter__方法。
    2.具有__exit__方法，当with代码组结束时自动调用。
    3.（非必要）利用__init__方法完成初始化

In [1]:
# DB.py
# import mysql.connector

class UseDatabase:
    """创建一个连接、断开数据库的上下文管理器"""
    def __init__(self, config: dict) -> None:
        self.config = config

    def __enter__(self) -> 'cursor':
        """建立与数据库的连接，并返回游标对象"""
        self.conn = mysql.connector.connect(**self.config)
        self.cursor = self.conn.cursor()
        return self.cursor

    def __exit__(self, exc_type, exc_val, exc_tb) -> None:
        """断开与数据库的连接，完成清理工作"""
        self.conn.commit()
        self.conn.close()
        self.cursor.close()

In [ ]:
# web.py
# from DB.py import UseDatabase

def log_request(req, res):
    """将查询信息写入数据库"""
    with UseDatabase(app.config['dbconfig']) as cursor:  # 自动初始化__init__，调用__enter__方法
        _SQL = '''insert into log
                    (phrase, letters, ip, browser_string, results)
                    values 
                    (%s, %s, %s, %s, %s)'''

        cursor.execute(_SQL, (req.form['phrase'],
                              req.form['letters'],
                              req.remote_addr,
                              req.user_agent.browser,
                              res))
    
    # with代码组结束后无需手动断开连接
    # 自动调用__exit__方法

In [ ]:
d = {}

d['xiaomi'] = 300
